# Titanic disaster model

### Inrotuction
#### to do:
- briefly describe model
- talk about recent methods used for problems like this\
- talk about methods we want to use
- what do we expect

https://www.kaggle.com/competitions/titanic

https://www.kaggle.com/code/alexisbcook/titanic-tutorial/notebook

https://www.overleaf.com/read/fqxnygwqtnjs#fbe3c8

### Seting up environment

In [22]:
import pandas as pd
import numpy as np
import matplotlib as plt
from scipy import optimize
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize, LinearConstraint

In [23]:
gh_path = 'https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/'
df_train = pd.read_csv(gh_path + 'train.csv')
df_test = pd.read_csv(gh_path + 'test.csv')
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### Feature engeneering
#### To do:
- change values of sex to binary (f.e. male = 0, female = 1),
- reduce number of values (drop columns: Name, Ticket, ?Fare?, Cabin, Embarked),
- drop keys with NaN values

A: Fare value can carry some information about Cabin, if we want to include passenger placement in model.

In [24]:
# Drop unnecessary columns
columns_to_drop = ['Name','Ticket', 'Cabin', 'Embarked']
df_train_processed = df_train.drop(columns=columns_to_drop)
df_test_processed = df_test.drop(columns=columns_to_drop)

# Drop keys with any NaN values
df_train_processed = df_train_processed.dropna()
df_test_processed = df_test_processed.dropna()

# Sex changed to binary
df_train_processed['Sex'] = df_train_processed['Sex'].replace({'male': 0, 'female': 1})
df_test_processed['Sex'] = df_test_processed['Sex'].replace({'male': 0, 'female': 1})

print(df_train_processed)

     PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare
0              1         0       3    0  22.0      1      0   7.2500
1              2         1       1    1  38.0      1      0  71.2833
2              3         1       3    1  26.0      0      0   7.9250
3              4         1       1    1  35.0      1      0  53.1000
4              5         0       3    0  35.0      0      0   8.0500
..           ...       ...     ...  ...   ...    ...    ...      ...
885          886         0       3    1  39.0      0      5  29.1250
886          887         0       2    0  27.0      0      0  13.0000
887          888         1       1    1  19.0      0      0  30.0000
889          890         1       1    0  26.0      0      0  30.0000
890          891         0       3    0  32.0      0      0   7.7500

[714 rows x 8 columns]


### Perceptron

#### To do:
- define weight vector W (imo randomly) and training set X
- DONE definition of perceptron class (linear, binary[if survived would be changed to -1 and 1] or/and LOGISTIC). That would include: activation function, loss function, fiting (gradient descent/newton-rap), all can be copied and slightly modified form collab "logistic_regression_and_svm",
- spliting training dataset and look for bad predictions,
- comparason with scikit-learn method

In [108]:
class PerceptronLogistic:
    def __init__(self, W : np.array,  b = 0, epo = 100, lr = 0.01):
        self.W = W
        self.N = len(W)
        self.b = b
        # how many learning iterations we give
        self.epo = epo
        # what is our step in the gradient
        self.lr = lr
    def get_N(self):
        return self.N
    
    def get_b(self):
        return self.b
    
    def get_W(self):
        return self.W
    
    ############################## SETTERS ##############################
    
    # Create setters for the perceptron
    def set_N(self, N):
        self.N = N
        self.W = np.zeros(N)
        
    def set_b(self, b):
        self.b = b
        
    def set_W(self, W : np.array):
        if W.ndim != 1:
            print("Cannot set such weights -> dimension wrong")
            return
        self.N = W.shape[0]
        self.W = W
    
    def set_lr(self, lr):
        self.lr = lr
        
    def set_epo(self, epo):
        self.epo = epo
    ############################## GETTERS OVERRIDE ##############################
    
    def __getitem__(self, key):
        return self.W[key]
    
    def __setitem__(self, key, value):
        self.W[key] = value
        
    def __getslice(self, i, j):
        return self.W[i:j]
    
    def get_lr(self):
        return self.lr

    def get_epo(self):
        return self.epo
    
    # set the string output of the perceptron 
    def __str__(self):
        return f"Am a perceptron of N={self.N} dimension{'s' if self.N > 1 else ''} biased with b={self.b}"    

    ############################## OPERATORS OVERRIDE ##############################
     
    def __mul__(self, other):
        return self.activation_function(other)
    
    def __rmul__(self, other):
        return self.__mul__(other)

    ############################## PERCEPTRON METHODS ##############################
    
    '''
    Net output is the basic body action of the perceptron. On top of it, the activation function is used.
    '''
    def net_output(self, X):
        return np.dot(X, self.W) + self.b

    def activation_function(self, X):
        net_output = self.net_output(X)
        return (1.0 / (1.0 + np.exp(-np.clip(net_output, -500, 500)))).reshape(-1, 1)

    def loss(self, y_true : np.array, y_pred : np.array, lambd=0.1):
            epsilon = 1e-15
            y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
            regularizer = (lambd / 2) * np.sum(self.W**2)
            return -np.sum(np.multiply(y_true, np.log(y_pred)) + np.multiply((1.0 - y_true), np.log(1.0 - y_pred))) + regularizer

    def gradient(self, x_true, y_true, prediction):
        val = y_true - prediction
        suma_w = np.dot(val, x_true)
        suma_b = np.sum(val)
        return suma_b, suma_w

    def predict(self, X):
        return self.activation_function(X)    

    def fit(self, X, y, randomstate = None, batch = 1, verbose = False):
        '''
        Fit function allows to obtain the (probably most?) correct weights for the perceptron via the gradient descent algorithm.
        '''
        
        if type(X) != np.ndarray:
            X = np.array(X)
        if type(y) != np.ndarray:
            y = np.array(y).reshape(-1,1)
            
        # give fit the parameter randomstate and whenever it is not None, the weights
        # are reset to be random normal - this ensures random starting point of gradient descent
        if randomstate is not None:
            self.W = np.random.normal(0.0, 0.1, self.N)
            self.b = np.random.normal(0.0, 1.0)
        
        # Save the history of the losses. Why?
        history = []
        # If we want to calculate the gradient in buckets (look for description of the batch)
        bucket_num = len(X) // batch
        # slice the data onto batches without shuffling (no stochasticity)
        slicing = lambda x, b: x[(b-1)*batch:b*batch]
        
        # iterate epochs
        for epo in range(self.epo):
            # iterate batches
            loss = 0.0
            for bin in range(1, bucket_num + 1):
                X_slice = slicing(X,bin)
                y_slice = slicing(y,bin)
                # predict the output for a given slice (what is the shape of the output?)
                pred = self.predict(X_slice)
                
                suma_b, suma_w = self.gradient(X_slice, y_slice, pred.flatten())

                # calculate loss
                loss += self.loss(y_slice, pred.flatten())
                
                # update the weights
                self.W += np.mean(suma_w) * self.lr
                self.b += np.mean(suma_b) * self.lr
            # calculate average loss
            loss/=bucket_num
            # if verbose:
                # print(f'epo:{epo}->loss={loss}')
            history.append(loss.flatten())
        return np.array(history).flatten()


    def plot_history(self, history, ax = None):
        '''
        Basic history plot
        '''        
        if ax is None:
            fig, ax = plt.subplots()
        ax.set_xlabel('epo')
        ax.set_ylabel('loss')
        ax.plot(history)

In [109]:
#skylearn attempt
X = df_train_processed.drop('Survived', axis=1)
y = df_train_processed['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [110]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

perceptron_model = PerceptronLogistic(W=np.zeros(X_train.shape[1]), b=0.1, epo=1000, lr=2e-3)
history = perceptron_model.fit(X_train, y_train, randomstate=42, batch=32, verbose=True)
y_pred = perceptron_model.predict(X_test)
y_pred_binary = (y_pred >= 0.5).astype(int)
accuracy = np.mean(y_pred_binary.flatten() == y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.3916083916083916


In [111]:
from sklearn.linear_model import LogisticRegression

# Use scikit-learn's Logistic Regression as a benchmark
lr_model = LogisticRegression()
lr_model.fit(X_train_scaled, y_train)
lr_accuracy = lr_model.score(X_test_scaled, y_test)
print(f"Scikit-learn Logistic Regression Accuracy: {lr_accuracy}")


Scikit-learn Logistic Regression Accuracy: 0.7552447552447552


### Suport Vector Machine
#### To do: 
- define 6D weight vector (Pclass,Sex,Age,SibSp,Parch,Fare), loss function (Lagrange function), calculate width of the margin (constraint equation), Optimize and get the values of $\vec{\alpha}$, w and bias b.  kernel transformation(s),
- soft- (leanring rate (regulatization) C) and hard-margin,
- spliting training dataset and look for bad predictions,
- compare with scikit-learn method.

$$ L(\vec{\alpha}) = \sum _i \alpha _i - \frac{1}{2} \sum _{i,j} \alpha _i \alpha _j y_i y_j \vec{X}_i\cdot \vec{X} _j. $$
Kuhn-Tucker theorem: the solution we find here will be the same as the solution to the original problem
##### [`REMARK`] It's crucial to keep track of this equation and the dot product within it, as it becomes highly significant later on, especially when dealing with nonlinear data.

In [12]:
Y = df_train_processed['Survived'].values
X = df_train_processed[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=420)

In [14]:
#sklearn for comparason
svm_model = SVC(kernel='linear')  #try different kernels
svm_model.fit(X_train, Y_train)

# Make predictions on the test set
predictions = svm_model.predict(X_test)

# Evaluate the model
accuracy = svm_model.score(X_test, Y_test)
print(f"Accuracy: {accuracy}")

# # save the results
# output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
# output.to_csv('SVM_kaggle_submission.csv', index=False)
# print("Your submission was successfully saved!")

Accuracy: 0.8333333333333334


NameError: name 'test_data' is not defined

#### Random Forest Model
##### To do:
- arono

##### Random forest (kaggle example)

In [1]:
from sklearn.ensemble import RandomForestClassifier
test_data = df_test
y = df_train["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(df_train[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('RF_kaggle_submission.csv', index=False)
print("Your submission was successfully saved!")

NameError: name 'df_test' is not defined

#### Analysis of results

JAK PCA TO MOŻNA BIBLIOTEK UŻYĆ 


#### Summary
To do:
- did we reachedour goals?
- what is interesting in our solutions
- what could be done better? limits and possible improvement